# Tutorial: Integrate two Xenium breast cancer samples

## Prepare the input .h5ad file
Download the Feature-cell Matrix (HDF5) and Cell summary file (CSV) from the Xenium breast cancer tumor microenvironment Dataset (https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast).

Then get the raw .h5ad file in jupyter with the following code

In [ ]:
! mkdir ./tutorial_data
! mkdir ./tutorial_data/xenium_data
! wget -P ./tutorial_data/xenium_data/ https://cf.10xgenomics.com/samples/xenium/preview/Xenium_FFPE_Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_cell_feature_matrix.h5
! wget -P ./tutorial_data/xenium_data/ https://cf.10xgenomics.com/samples/xenium/preview/Xenium_FFPE_Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv.gz

! wget -P ./tutorial_data/xenium_data/ https://cf.10xgenomics.com/samples/xenium/preview/Xenium_FFPE_Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_cell_feature_matrix.h5
! wget -P ./tutorial_data/xenium_data/ https://cf.10xgenomics.com/samples/xenium/preview/Xenium_FFPE_Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_cells.csv.gz

! gunzip ./tutorial_data/xenium_data/Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv.gz
! gunzip ./tutorial_data/xenium_data/Xenium_FFPE_Human_Breast_Cancer_Rep2_cells.csv.gz

In [ ]:
import scanpy as sc
import pandas as pd

for rep in [1, 2]:
    adata = sc.read_10x_h5(f"./tutorial_data/xenium_data/Xenium_FFPE_Human_Breast_Cancer_Rep{rep}_cell_feature_matrix.h5")
    df = pd.read_csv(f"./tutorial_data/xenium_data/Xenium_FFPE_Human_Breast_Cancer_Rep{rep}_cells.csv")
    df.index = adata.obs_names
    adata.obs = df.copy()
    adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].to_numpy()
    adata.write(f"./tutorial_data/xenium_data/P{rep}.h5ad")

## Preparation

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# the location of R (used for the mclust clustering)
import os
os.environ['R_USER'] =  '/usr/lib/R'

import numpy as np
import anndata as ad
import scipy.sparse as sp
import torch

import STIntg

used_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Load Data

In [ ]:
Batch_list = []
adj_list = []

section_ids = ['Rep1', 'Rep2']
for section_id in section_ids:
    print(section_id)
    adata = sc.read_h5ad(f"./tutorial_data/xenium_data/{section_id}.h5ad")
    adata.layers["counts"] = adata.X.copy()

    # make spot name unique
    adata.obs_names = [x + '_' + section_id for x in adata.obs_names]

    STIntg.Cal_Spatial_Net(adata, rad_cutoff=20)

    # Normalization
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    # sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000) #ensure enough common HVGs in the combined matrix
    # adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)


## Concat the scanpy objects for multiple slices

In [ ]:
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')
print('adata_concat.shape: ', adata_concat.shape)
adata_concat.uns['edgeList'] = STIntg.adj_concat(adj_list)

## Running STIntg

In [ ]:
spatial_net_args = {'rad_cutoff': 20, 'model': 'Radius', 'verbose': True}
adata_concat = STIntg.train(adata_concat, verbose=True, knn_neigh=50, num_batch=2,
                                device=used_device, batch_data=True, hidden_dims=[256, 128],
                                spatial_net_args=spatial_net_args,
                                pretrain_epochs=200, n_epochs=400)

## Clustering

In [ ]:
sc.pp.neighbors(adata_concat, use_rep='STIntg')
sc.tl.umap(adata_concat)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = "DejaVu Sans"
plt.rcParams["figure.figsize"] = (3, 3)
plt.rcParams['font.size'] = 10

In [ ]:
for res in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    sc.tl.louvain(adata_concat,  resolution=res, key_added=f'louvain_{res}')

In [ ]:
# adata_concat.write_h5ad('/data1/st_data/xenium_data/Xenium_STIntg.h5ad')
adata_concat = sc.read_h5ad('/data1/st_data/xenium_data/Xenium_STIntg.h5ad')

In [ ]:
sc.pl.umap(adata_concat, color=['batch_name', 'louvain_0.7'], show=True, wspace=0.5)

In [ ]:
section_ids = ['Rep1', 'Rep2']
Batch_list = []
for section_id in section_ids:
    Batch_list.append(adata_concat[adata_concat.obs['batch_name'] == section_id])


In [ ]:
plt.rcParams["figure.figsize"] = (8, 6)
sc.pl.embedding(Batch_list[0],color='louvain_0.7', title=section_ids[0], basis="spatial",
                legend_fontsize=12, show=False, frameon=False, size=3)
sc.pl.embedding(Batch_list[1], color='louvain_0.7', title=section_ids[1], basis="spatial",
                legend_fontsize=12, show=False, frameon=False, size=3)